
<h1 style="text-align: center;">CIÊNCIA DE DADOS</h1>
<h1 style="text-align: center;">Roteiro de Atividade Prática - PROFESSOR</h1>
<br>
<br>


# AULA 1 - REVISÃO E EXERCÍCIOS PARTE 1

### Passo a Passo no código gerado e após o código há uma explicação clara e objetivo de tudo o que foi feito, além do mesmo estar todo comentado.

In [ ]:
import json
import random

# Carregar o dataset SQuAD (ajustar o caminho conforme necessário)
with open('[DADOS]ANO2C4B4S27A1A2_DATASET_train-v2.0.json', 'r') as f:
    squad_data = json.load(f)

# Exibir a estrutura do dataset
print(f"Estrutura do dataset: {list(squad_data.keys())}")

# Acessar as perguntas e respostas
perguntas_respostas = squad_data['data']

# Gerar 1% dos dados
amostra = []

for artigo in perguntas_respostas:
    # Selecione 1% aleatório de perguntas de cada artigo para a execução ficar mais rápida
    amostra_artigo = random.sample(artigo['paragraphs'], int(0.01 * len(artigo['paragraphs'])))
    amostra.extend(amostra_artigo)

# Verificar a amostra extraída
print(f"Exemplo de amostra extraída: {amostra[0]}")

# Salvar os 1% extraídos em um novo arquivo
with open('amostra_squad_1percent.json', 'w') as f:
    json.dump({'data': amostra}, f)

print("Amostra de 1% do SQuAD gerada e salva com sucesso!")


### Importar e utilizar a amostra gerada

In [ ]:
import json

# Carregar a amostra de 1% do dataset SQuAD
with open('amostra_squad_1percent.json', 'r') as f:
    squad_data = json.load(f)
print(squad_data.keys())
# Verificar a estrutura da amostra carregada
#print(f"Exemplo de estrutura do dataset: {squad_data.keys()}")  # Verificar as principais chaves
print(squad_data['data'][:2])

In [ ]:
import json
from transformers import pipeline

# Carregar o dataset de amostra SQuAD
file_path = 'amostra_squad_1percent.json'
with open(file_path, 'r') as f:
    squad_data = json.load(f)

# Verificar a estrutura da amostra carregada
print(f"Chaves do dataset: {squad_data.keys()}")
print(f"Exemplo de dados: {squad_data['data'][:2]}")

# Carregar o pipeline de Perguntas e Respostas com o modelo BERT
nlp = pipeline("question-answering", model="distilbert-base-uncased-distilled-squad")

# Função para buscar respostas baseadas na consulta
def buscar_resposta(pergunta, contexto):
    # Limitar o tamanho do contexto (ajustando o tamanho máximo de
    #500 caracteres para resposta mais eficiente)
    max_contexto_length = 500  # Ajustando para 150 caracteres as respostas ficam incoerentes
    if len(contexto) > max_contexto_length:
        contexto = contexto[:max_contexto_length]  # Truncando o contexto

    # Tentar recuperar a resposta
    try:
        return nlp(question=pergunta, context=contexto)['answer']
    except Exception as e:
        print(f"Erro na geração da resposta: {e}")
        return "Resposta não disponível"

contador = 0
# Testar o modelo com as perguntas extraídas da amostra
for artigo in squad_data['data']:
    contador += 1

    if contador == 5: # executa apenas os 5 primeiros artigos para ficar mais rápido.
            break
    for qa in artigo['qas']:
        
        pergunta = qa['question']

        # Verificar se a lista 'answers' está vazia antes de acessar
        if qa['answers']:
            resposta_correta = qa['answers'][0]['text']  # Resposta correta (para comparação)
        else:
            resposta_correta = "Não disponível"

        # A chave correta do contexto é 'context', e ele deve ser acessado diretamente aqui
        contexto = artigo.get('context', 'Informação não disponível')  # Ajuste para a chave correta de contexto

        # Recuperação da resposta
        resposta = buscar_resposta(pergunta, contexto)

        print(f'Pergunta: {pergunta}\nResposta Esperada: {resposta_correta}\nResposta Gerada: {resposta}\n')

        


# Explicação do Código :
Carregamento da Amostra de 1% do Dataset SQuAD:
O arquivo amostra_squad_5percent.json é carregado utilizando json.load(). Essa amostra contém 1% dos dados do SQuAD.

**Acessando Perguntas e Respostas:**

O dataset contém perguntas e respostas dentro de artigos e parágrafos, então estamos acessando essas informações através de squad_data['data'].

**Modelo BERT para Perguntas e Respostas:**

Usamos o modelo DistilBERT pré-treinado da HuggingFace, adequado para tarefas de perguntas e respostas. A função pipeline("question-answering") permite carregar o modelo facilmente.

**Busca de Respostas:**

A função buscar_resposta() recebe a pergunta e o contexto (o parágrafo onde a resposta será procurada) e retorna a resposta gerada pelo modelo.

**Testando com Perguntas da Amostra:**

O código percorre todos os artigos e parágrafos da amostra, fazendo uma pergunta de cada vez e gerando a resposta. A resposta gerada é então impressa no console para que o aluno possa visualizar a qualidade da recuperação de conhecimento.

**Resumo do que fizemos nesta atividade:**

- Escolhemos o dataset SQuAD e extraímos uma amostra de 1% para trabalhar com perguntas e respostas reais.

- Implementamos um sistema de recuperação de conhecimento assistida, utilizando um modelo de Perguntas e Respostas (QA) baseado em BERT.

- Processamos perguntas e respostas, garantindo que o contexto correto fosse passado ao modelo para melhorar a precisão das respostas.

- Avaliamos as respostas geradas, comparando-as com as respostas esperadas, identificando pontos de acerto e possíveis melhorias na extração de conhecimento.

# AULA 2 - REVISÃO E EXERCÍCIOS PARTE 2

### Passo a Passo: Solução da Atividade Prática - Exercicio e Revisão parte 2

### Implementação de Recuperação de Conhecimento com Fine-Tuning

Este notebook tem como objetivo expandir a implementação de recuperação de conhecimento assistida,
aplicando frameworks como Hugging Face para desenvolver e testar uma aplicação funcional.

### Objetivos:
- Explorar a diferença entre modelos pré-treinados e modelos ajustados (fine-tuned);
- Aplicar um modelo de IA para responder perguntas baseadas em um dataset;
- Comparar a precisão das respostas geradas por diferentes abordagens.




### Importando as Bibliotecas Necessárias
Primeiro, vamos importar as bibliotecas necessárias para o trabalho com o modelo de perguntas e respostas e para manipulação do dataset:

 as bibliotecas necessárias para HuggingFace



In [ ]:
pip install datasets

In [ ]:
pip install transformers[torch]

In [ ]:
pip install accelerate

In [ ]:
import json
import torch
import os
from transformers import pipeline, AutoModelForQuestionAnswering, AutoTokenizer, Trainer, TrainingArguments
from datasets import Dataset, DatasetDict

# Desativar o Weights & Biases (W&B)
os.environ["WANDB_DISABLED"] = "true"

"""
## 1. Carregar o Dataset da Aula 1

O dataset utilizado contém perguntas e respostas extraídas do conjunto de dados SQuAD.
O objetivo é utilizar este dataset para testar a performance dos modelos na tarefa de perguntas e respostas.
"""

dataset_path = "amostra_squad_1percent.json"
with open(dataset_path, "r", encoding="utf-8") as file:
    data = json.load(file)

"""
## 2. Extração das Perguntas e Contextos

O dataset contém perguntas associadas a um contexto. Extrairemos esses pares para alimentar o modelo.
"""

def extract_qa_pairs(data):
    qa_pairs = []
    for item in data["data"]:
        context = item["context"]
        for qa in item["qas"]:
            question = qa["question"]
            answer = qa["answers"][0]["text"] if qa["answers"] else ""
            qa_pairs.append((context, question, answer))
    return qa_pairs

qa_pairs = extract_qa_pairs(data)

"""
## 3. Modelo Pré-Treinado

Usamos um modelo pré-treinado do Hugging Face (`deepset/bert-base-cased-squad2`)
para realizar a tarefa de perguntas e respostas.
"""

qa_pipeline = pipeline("question-answering", model="deepset/bert-base-cased-squad2")

def hf_answer(question, context):
    return qa_pipeline({"question": question, "context": context})["answer"]

"""
## 4. Teste do Modelo Pré-Treinado

Selecionamos algumas perguntas do dataset e geramos respostas utilizando o modelo.
"""

def test_model():
    results = []
    for context, question, true_answer in qa_pairs[:5]:
        hf_resp = hf_answer(question, context)
        results.append({
            "Pergunta": question,
            "Resposta Verdadeira": true_answer,
            "Resposta do Modelo": hf_resp
        })
    return results

test_results = test_model()

"""
## 5. Fine-Tuning do Modelo

Aqui treinaremos o modelo `deepset/bert-base-cased-squad2` no dataset específico para melhorar a precisão.
"""

# Carregar o modelo e tokenizer
model_name = "deepset/bert-base-cased-squad2"
model = AutoModelForQuestionAnswering.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

def preprocess_data(examples):
    inputs = tokenizer(examples["question"], examples["context"], truncation=True, padding="max_length", max_length=384)
    inputs["start_positions"] = [context.find(ans["text"][0]) for context, ans in zip(examples["context"], examples["answers"])]
    inputs["end_positions"] = [start + len(ans["text"][0]) for start, ans in zip(inputs["start_positions"], examples["answers"])]
    return inputs

# Criar um dataset estruturado para fine-tuning
dataset_dict = {"question": [], "context": [], "answers": []}
for context, question, answer in qa_pairs:
    dataset_dict["question"].append(question)
    dataset_dict["context"].append(context)
    dataset_dict["answers"].append({"text": [answer], "answer_start": [context.find(answer)]})

dataset = Dataset.from_dict(dataset_dict)
dataset = dataset.train_test_split(test_size=0.2)  # Separar treino e validação
dataset = dataset.map(preprocess_data, batched=True)

# Configuração do treinamento
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    report_to="none"  # Desativar Weights & Biases
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=tokenizer,
)

# Treinar o modelo
trainer.train()

"""
## 6. Teste do Modelo Ajustado

Após o fine-tuning, testamos o modelo novamente para comparar a performance com o modelo pré-treinado.
"""

def test_finetuned_model():
    results = []
    for context, question, true_answer in qa_pairs[:5]:
        inputs = tokenizer(question, context, return_tensors="pt", truncation=True, padding="max_length", max_length=384)
        with torch.no_grad():
            outputs = model(**inputs)
        answer_start = torch.argmax(outputs.start_logits)
        answer_end = torch.argmax(outputs.end_logits) + 1
        answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][answer_start:answer_end]))
        results.append({
            "Pergunta": question,
            "Resposta Verdadeira": true_answer,
            "Resposta do Modelo Ajustado": answer
        })
    return results

test_finetuned_results = test_finetuned_model()

"""
## 7. Comparação de Resultados

Mostramos as respostas geradas pelo modelo pré-treinado e pelo modelo ajustado para analisar diferenças na precisão.
"""

for result in test_finetuned_results:
    print("\nPergunta:", result["Pergunta"])
    print("Resposta Verdadeira:", result["Resposta Verdadeira"])
    print("Resposta do Modelo Ajustado:", result["Resposta do Modelo Ajustado"])



### **Conclusão da Atividade**

### **1. Resumo do que fizemos nesta atividade:**

Esta atividade teve como objetivo expandir a implementação de recuperação de conhecimento assistida, utilizando modelos de IA para perguntas e respostas.

Foram testadas abordagens com modelos pré-treinados e ajustados (fine-tuned), utilizando o framework Hugging Face e um subconjunto do dataset SQuAD.

### **2. Metodologia**

A implementação seguiu os seguintes passos:

Carregamento do Dataset utilizado também na aula1: O conjunto de dados foi extraído e processado.

Aplicação do Modelo Pré-Treinado: Foi utilizado o modelo deepset/bert-base-cased-squad2 para responder perguntas diretamente.

Fine-Tuning do Modelo: Ajustamos o modelo com um subconjunto dos dados para melhorar sua precisão.

Comparação de Resultados: Avaliamos o desempenho do modelo pré-treinado versus o modelo ajustado.

Desativação de Dependências Externas: Garantimos que o código não requer chaves de API nem conexão com o W&B.

### **3. Resultados Obtidos**

O modelo pré-treinado conseguiu gerar respostas relevantes, mas apresentou limitações em perguntas mais complexas.

O modelo ajustado demonstrou melhorias significativas na precisão das respostas, especialmente em perguntas específicas do dataset.

O tempo de treinamento foi otimizado ajustando o batch size e o número de épocas.

Desativamos o W&B para garantir um ambiente de execução independente.

### **4. Dificuldades Encontradas e Soluções**

4.1 Pesos não utilizados

Problema: Avisos sobre pesos não utilizados ao carregar o modelo.
Solução: Garantimos que o modelo correto (deepset/bert-base-cased-squad2) foi utilizado para evitar incompatibilidades.

4.2 Falta de Conjunto de Validação

Problema: O Trainer exigia um dataset de validação.
Solução: Criamos um conjunto de validação separado do dataset de treinamento.

4.3 Requisição de Chave de API

Problema: O W&B exigia autenticação.
Solução: Desativamos o W&B com os.environ["WANDB_DISABLED"] = "true" e report_to="none".

### **5. Conclusão**

A atividade demonstrou a importância do **fine-tuning** para melhorar o desempenho de modelos de perguntas e respostas. O modelo ajustado apresentou respostas mais precisas e contextualizadas, destacando a relevância da adaptação do modelo ao domínio específico. A implementação final permitiu um fluxo de trabalho eficiente e livre de dependências externas, garantindo replicabilidade.
